In [14]:
import os 
import matplotlib as plt
import PIL as Image 
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers , Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


import pandas as pd
import cv2

import tifffile

import imageio.v3 as iio
import matplotlib.pyplot as plt

In [15]:
train_image_dir=r"D:\cellula\task_2\data\images"
train_mask_dir=r"D:\cellula\task_2\data\labels"

In [16]:
train_img_files=sorted(os.listdir(train_image_dir))
train_mask_files=sorted(os.listdir(train_mask_dir))

In [17]:
images= [os.path.join(train_image_dir, file) for file in os.listdir(train_image_dir)]
masks= [os.path.join(train_mask_dir, file) for file in os.listdir(train_mask_dir)]

data = pd.DataFrame({"image":images, "mask":masks})
train_df , val_df =train_test_split(data , train_size=0.8 ,shuffle=True , random_state=123)

In [18]:
image_datagen=ImageDataGenerator(rescale=1/255)
mask_datagen=ImageDataGenerator(rescale=1/255)

In [19]:
def create_generator(image_generator, mask_generator, batch_size=16):
    while True:
        for (img_batch,mask_batch) in zip(image_generator,mask_generator):
            yield img_batch, mask_batch

In [20]:
train_image_generator = image_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col ="image",
    y_col=None,
    target_size=(128,128),
    batch_size=16,
    class_mode=None,
    color_mode="rgb",
    shuffle=True,
    seed=42
)

train_mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col ="mask",
    y_col=None,
    target_size=(128,128),
    batch_size=16,
    class_mode=None,
    color_mode="grayscale",
    shuffle=True,
    seed=42   
)


train_generator = create_generator(train_image_generator, train_mask_generator)
# or same use  val_generator = zip(val_image_generator, val_mask_generator)

Found 244 validated image filenames.
Found 244 validated image filenames.


In [21]:
val_image_generator=image_datagen.flow_from_dataframe(
    dataframe = val_df,
    x_col ="image",
    y_col=None,
    target_size=(128,128),
    batch_size=16,
    class_mode=None,
    color_mode="rgb",
    shuffle=False,
    seed=42
)

val_mask_generator=mask_datagen.flow_from_dataframe(
    dataframe = val_df,
    x_col ="mask",
    y_col=None,
    target_size=(128,128),
    batch_size=16,
    class_mode=None,
    color_mode="grayscale",
    shuffle=False,
    seed=42
)

val_generator = create_generator(val_image_generator, val_mask_generator)

Found 62 validated image filenames.
Found 62 validated image filenames.


In [22]:
def unet_model(input_size=(128,128)):
    inputs =Input(input_size)


    c1 =layers.Conv2D(64, (3,3), activation="relu", padding="same")(inputs)
    c1 =layers.Conv2D(64, (3,3), activation="relu", padding="same")(c1)
    p1 =layers.MaxPooling2D( (2,2) )(c1)

    c2 =layers.Conv2D(128, (3,3), activation="relu", padding="same")(p1)
    c2 =layers.Conv2D(128, (3,3), activation="relu", padding="same")(c2)
    p2 =layers.MaxPooling2D( (2,2) )(c2)

    c3 =layers.Conv2D(256, (3,3), activation="relu", padding="same")(p2)
    c3 =layers.Conv2D(256, (3,3), activation="relu", padding="same")(c3)
    p3 =layers.MaxPooling2D( (2,2) )(c3)

    c4 =layers.Conv2D(512, (3,3), activation="relu", padding="same")(p3)
    c4 =layers.Conv2D(512, (3,3), activation="relu", padding="same")(c4)
    p4 =layers.MaxPooling2D( (2,2) )(c4)   


    c5 =layers.Conv2D(1024, (3,3), activation="relu", padding="same")(p4)
    c5 =layers.Conv2D(1024, (3,3), activation="relu", padding="same")(c5)
  
 # decoder

    u6 =layers.Conv2DTranspose(512, (2,2), strides=(2,2), padding="same")(c5)
    #u6 =layers.Concatenate([u6 , c4])
    u6 = layers.Concatenate()([u6, c4])
    c6 =layers.Conv2D(512, (3,3), activation="relu", padding="same")(u6)
    c6 =layers.Conv2D(512, (3,3), activation="relu", padding="same")(c6)

    u7 =layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c6)
    #u7 =layers.Concatenate([u7 , c3])
    u7 = layers.Concatenate()([u7, c3])
    c7 =layers.Conv2D(256, (3,3), activation="relu", padding="same")(u7)
    c7 =layers.Conv2D(256, (3,3), activation="relu", padding="same")(c7)

    u8 =layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c7)
    #u8 =layers.Concatenate([u8 , c2])
    u8 = layers.Concatenate()([u8, c2])
    c8 =layers.Conv2D(128, (3,3), activation="relu", padding="same")(u8)
    c8 =layers.Conv2D(128, (3,3), activation="relu", padding="same")(c8)

    u9 =layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c8)
    #u9 =layers.Concatenate([u9 , c1])
    u9 = layers.Concatenate()([u9, c1])
    c9 =layers.Conv2D(64, (3,3), activation="relu", padding="same")(u9)
    c9 =layers.Conv2D(64, (3,3), activation="relu", padding="same")(c9)  

    #output layer


    outputs= layers.Conv2D(1, (1,1), activation="sigmoid")(c9)


    model = Model(inputs=[inputs], outputs= [outputs])

    return model  

In [23]:
model = unet_model(input_size=(128, 128, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [24]:
history = model.fit(
train_generator,
validation_data=val_generator,
epochs=5,
steps_per_epoch=len(train_df)//16,
validation_steps=len(val_df) //16

)

More samples per pixel than can be decoded: 12


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002B86EE79DA0>